### Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.graphics.tsaplots as tsaplots
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.statespace.sarimax import SARIMAX
import itertools

from sklearn.metrics import mean_squared_error

### Get Data

In [2]:
train = pd.read_csv('./data/train.csv')

'''Setting up the log transformed and differenced timeseries of daily sales across all stores. This step is the
result of the exploratory data analysis.'''

dailysales = np.log(train.groupby('date')['sales'].sum()).diff(7)
dailysales = dailysales.dropna()

# Explicitly setting the DatetimeIndex frequency
dailysales.index = pd.DatetimeIndex(dailysales.index.values, freq = 'D')

dailysales.head()

2013-01-08   -0.009980
2013-01-09    0.014516
2013-01-10    0.010573
2013-01-11   -0.002811
2013-01-12    0.026858
Freq: D, Name: sales, dtype: float64

### SARIMA Model Parameters - Grid Search

The exploratory data analysis helps us set up the following grid for the transformed timeseries:
<ol>
    <li>MA params: p = 1, 2, 3, 4, 5, 6 and P = 0,
    <li>AR params: q = 1, 2, 3, 4, 5, 6 and Q = 7, 8,
    <li>d = D = 0, since we have already differenced the series, and
    <li>s = 7.
</ol>

In [3]:
# Set the parameter grid
p = range(1,7)
d = range(0,1)
q = range(1, 7)
P = range(0,1)
D = range(0,1)
Q = range(7, 9)
s = range(7,8)

# Forming the grid
pdq = list(itertools.product(p, d, q))
seasonal_pdq = list(itertools.product(P, D, Q, s))

# SARIMA model pipeline
for param in pdq:
    for seasonal_param in seasonal_pdq:
        try:
            model = SARIMAX(dailysales, order = param, seasonal_order = seasonal_param)
            results = model.fit(method = 'powell')
            print('SARIMA{},{} - AIC:{}'.format(param, seasonal_param, results.aic))
        except:
            continue


Optimization terminated successfully.
         Current function value: -2.201292
         Iterations: 8
         Function evaluations: 941
SARIMA(1, 0, 1),(0, 0, 7, 7) - AIC:-7988.300057785762
Optimization terminated successfully.
         Current function value: -2.201308
         Iterations: 8
         Function evaluations: 1021
SARIMA(1, 0, 1),(0, 0, 8, 7) - AIC:-7986.3601716889025
Optimization terminated successfully.
         Current function value: -2.201191
         Iterations: 8
         Function evaluations: 1017
SARIMA(1, 0, 2),(0, 0, 7, 7) - AIC:-7985.932132235948
Optimization terminated successfully.
         Current function value: -2.201216
         Iterations: 8
         Function evaluations: 1098
SARIMA(1, 0, 2),(0, 0, 8, 7) - AIC:-7984.025308720158
Optimization terminated successfully.
         Current function value: -2.201343
         Iterations: 9
         Function evaluations: 1242
SARIMA(1, 0, 3),(0, 0, 7, 7) - AIC:-7984.485174787294
Optimization terminated succes

/home/sanjeev/anaconda3/envs/timeseries/lib/python3.7/site-packages/statsmodels/tsa/statespace/sarimax.py:963: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/home/sanjeev/anaconda3/envs/timeseries/lib/python3.7/site-packages/statsmodels/tsa/statespace/sarimax.py:975: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


Optimization terminated successfully.
         Current function value: -2.201003
         Iterations: 7
         Function evaluations: 909
SARIMA(2, 0, 1),(0, 0, 7, 7) - AIC:-7985.249233830925
Optimization terminated successfully.
         Current function value: -2.201022
         Iterations: 7
         Function evaluations: 979
SARIMA(2, 0, 1),(0, 0, 8, 7) - AIC:-7983.317911738333
Optimization terminated successfully.
         Current function value: -2.201298
         Iterations: 7
         Function evaluations: 987
SARIMA(2, 0, 2),(0, 0, 7, 7) - AIC:-7984.3206898118515
Optimization terminated successfully.
         Current function value: -2.201313
         Iterations: 7
         Function evaluations: 1055
SARIMA(2, 0, 2),(0, 0, 8, 7) - AIC:-7982.377818122637
Optimization terminated successfully.
         Current function value: -2.201309
         Iterations: 7
         Function evaluations: 1049
SARIMA(2, 0, 3),(0, 0, 7, 7) - AIC:-7982.361773276102
Optimization terminated successf

SARIMA(5, 0, 4),(0, 0, 7, 7) - AIC:-7967.821679599426
Optimization terminated successfully.
         Current function value: -2.201526
         Iterations: 6
         Function evaluations: 1249
SARIMA(5, 0, 4),(0, 0, 8, 7) - AIC:-7973.1505548841305
Optimization terminated successfully.
         Current function value: -2.201185
         Iterations: 8
         Function evaluations: 1670
SARIMA(5, 0, 5),(0, 0, 7, 7) - AIC:-7971.910661926057
Optimization terminated successfully.
         Current function value: -2.201417
         Iterations: 8
         Function evaluations: 1750
SARIMA(5, 0, 5),(0, 0, 8, 7) - AIC:-7970.753777778279
Optimization terminated successfully.
         Current function value: -2.206336
         Iterations: 5
         Function evaluations: 1111
SARIMA(5, 0, 6),(0, 0, 7, 7) - AIC:-7988.650615298253
Optimization terminated successfully.
         Current function value: -2.206336
         Iterations: 5
         Function evaluations: 1157
SARIMA(5, 0, 6),(0, 0, 8, 7) 

Best results SARIMA(1, 0, 6),(0, 0, 7, 7) - AIC:-7996.116078993578

In [ ]:
train_len = np.int(0.9*len(tot_sls_trans))
train_tot_sls_trans = tot_sls_trans[:train_len]
test_tot_sls_trans = tot_sls_trans[train_len:]

In [ ]:
# Set the implied frequency of the timeseries to suppress a subsequent warning by SARIMAX()
dailysales_all.index = pd.DatetimeIndex(dailysales_all.index.values, 
                                           freq=dailysales_all.index.inferred_freq)

model = SARIMAX(dailysales_all, order = (1, 1, 1), seasonal_order = (1, 1, 1, 7), 
                trend = 'n').fit(method = 'powell')
res = model.resid

display(model.summary())

fig, (ax1, ax2) = plt.subplots(nrows = 2, figsize = (13, 9), sharex = True)

tsaplots.plot_pacf(res, ax = ax1, 
                   title = 'Partial Autocorrelation Function for Residues')

tsaplots.plot_acf(res, ax = ax2,
                  title = 'Autocorrelation Function for Residues')
ax2.set_xticks(np.arange(0, 36))
ax2.set_xlabel('Lags in Days', fontsize = '12', fontweight = 'bold')

plt.show()